In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np 
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv('diabetes.csv')
df=df.drop(columns='family_histroy')
df.replace(4.860753, np.nan, inplace=True)
df.dropna(inplace=True)
df["smoking"] = df["smoking"].replace({1.0: 0, 2.0: 0, 3.0: 1})
df_copy=df
df_copy["smoking"] = df["smoking"].astype(int)


X = df_copy.drop('smoking', axis=1)
y = df_copy['smoking']

In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [ ]:

class Tabular1DCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super(Tabular1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool(x).squeeze(-1)  # shape: (batch_size, channels)
        x = self.fc(x)
        return x

In [ ]:

model = Tabular1DCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")


model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = torch.argmax(predictions, dim=1)
    print(classification_report(y_test_tensor, predicted_classes))

Epoch 1/20 - Loss: 0.7698
Epoch 2/20 - Loss: 0.7613
Epoch 3/20 - Loss: 0.7531
Epoch 4/20 - Loss: 0.7452
Epoch 5/20 - Loss: 0.7376
Epoch 6/20 - Loss: 0.7302
Epoch 7/20 - Loss: 0.7230
Epoch 8/20 - Loss: 0.7161
Epoch 9/20 - Loss: 0.7093
Epoch 10/20 - Loss: 0.7027
Epoch 11/20 - Loss: 0.6962
Epoch 12/20 - Loss: 0.6899
Epoch 13/20 - Loss: 0.6837
Epoch 14/20 - Loss: 0.6776
Epoch 15/20 - Loss: 0.6716
Epoch 16/20 - Loss: 0.6657
Epoch 17/20 - Loss: 0.6599
Epoch 18/20 - Loss: 0.6541
Epoch 19/20 - Loss: 0.6484
Epoch 20/20 - Loss: 0.6428
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       189
           1       0.71      1.00      0.83       463

    accuracy                           0.71       652
   macro avg       0.36      0.50      0.42       652
weighted avg       0.50      0.71      0.59       652



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
